# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from g_keys import key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
#Load dataframe from file created before
city_data_df=pd.read_csv("../WeatherPy/city_data.csv")
city_data_df.head(25)

City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0              madimba -10.4000   40.3333     72.14        89          74   
1               muisne   0.6000  -80.0333     76.95        82          89   
2                buala  -8.1450  159.5921     84.34        73          91   
3           butaritari   3.0707  172.7902     82.20        80         100   
4             bengkulu  -3.8004  102.2655     76.60        84          63   
5               hobart -42.8794  147.3294     56.23        76           0   
6          tuktoyaktuk  69.4541 -133.0374     37.13        71          89   
7               albany  42.6001  -73.9662     65.32        92         100   
8            cape town -33.9258   18.4232     61.11        64           0   
9   severo-yeniseyskiy  60.3725   93.0408     52.14        94         100   
10              zanjan  36.6736   48.4787     68.40        26           0   
11              nemuro  43.3236  145.5750     49.15        99         100   
12           abu kamal  34.4506   40.9171     75.69        26           0   
13               bluff -46.6000  168.3333     49.80        80          93   
14             ushuaia -54.8000  -68.3000     26.26        86          20   
15         east london -33.0153   27.9116     60.60        87          87   
16        quatre cocos -20.2078   57.7625     76.08        73          64   
17             ahipara -35.1667  173.1667     60.19        65         100   
18           mahebourg -20.4081   57.7000     76.24        71          73   
19             qaanaaq  77.4840  -69.3632     27.05        85          99   
20           carnarvon -24.8667  113.6333     53.67        87         100   
21         clyde river  70.4692  -68.5914     30.81        86         100   
22             dunedin -45.8742  170.5036     56.80        62           9   
23          valleyview  55.0667 -117.2859     79.86        44          89   
24           jamestown  42.0970  -79.2353     67.59        68          90   

    Wind Speed Country        Date  
0         5.84      TZ  1622675485  
1         7.72      EC  1622675485  
2         5.91      SB  1622675485  
3        13.94      KI  1622675486  
4         4.54      ID  1622675486  
5         1.99      AU  1622675322  
6        10.09      CA  1622675471  
7         1.97      US  1622675424  
8         3.00      ZA  1622675320  
9         3.44      RU  1622675487  
10        6.06      IR  1622675407  
11        5.44      JP  1622675487  
12       15.39      SY  1622675488  
13        9.35      NZ  1622675488  
14        5.75      AR  1622675474  
15       15.14      ZA  1622675488  
16       18.48      MU  1622675489  
17        4.47      NZ  1622675489  
18       18.37      MU  1622675489  
19        3.38      GL  1622675489  
20        8.05      AU  1622675490  
21        3.27      CA  1622675490  
22        1.99      NZ  1622675490  
23        4.85      CA  1622675491  
24       11.50      US  1622675253

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=key)

In [67]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_df =city_data_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:

#Filter ideal weather conditions
ideal_cities = city_data_df[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 80)]
ideal_cities = ideal_cities[ideal_cities['Wind Speed'] < 10]
ideal_cities = ideal_cities[ideal_cities['Cloudiness'] == 0]

ideal_cities

City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
191              asmar  35.0333  71.3581     72.27        48           0   
273       saint-pierre -21.3393  55.4781     70.57        74           0   
295            dasoguz  41.8363  59.9666     76.95        47           0   
298               hami  42.8000  93.4500     73.60         9           0   
340             kerman  30.2832  57.0788     73.26        17           0   
459            shahrud  36.4182  54.9763     78.35        14           0   
520  sao joao da barra -21.6403 -41.0511     72.54        86           0   

     Wind Speed Country        Date  
191        7.72      AF  1622675540  
273        5.75      RE  1622675564  
295        8.86      TM  1622675571  
298        8.46      CN  1622675572  
340        3.67      IR  1622675584  
459        4.81      IR  1622675619  
520        9.80      BR  1622675637

191    9
273    9
295    9
298    9
340    9
459    9
520    9
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
#add fields for hotel infor
hotel_df = ideal_cities

hotel_df['Hotel Name'] = ""
hotel_df['Hotel City'] = ""
hotel_df['Hotel Country'] = ""
hotel_df

City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
191              asmar  35.0333  71.3581     72.27        48           0   
273       saint-pierre -21.3393  55.4781     70.57        74           0   
295            dasoguz  41.8363  59.9666     76.95        47           0   
298               hami  42.8000  93.4500     73.60         9           0   
340             kerman  30.2832  57.0788     73.26        17           0   
459            shahrud  36.4182  54.9763     78.35        14           0   
520  sao joao da barra -21.6403 -41.0511     72.54        86           0   

     Wind Speed Country        Date Hotel Name Hotel City Hotel Country  
191        7.72      AF  1622675540                                      
273        5.75      RE  1622675564                                      
295        8.86      TM  1622675571                                      
298        8.46      CN  1622675572                                      
340        3.67      IR  1622675584                                      
459        4.81      IR  1622675619                                      
520        9.80      BR  1622675637

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [62]:
for index, hotel in hotel_df.iterrows():
    try:        
        query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "type": "hotel",
        "radius": 5000,
         "rankby": "distance",
        "key": key,
        }

        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"

        response = requests.get(query_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"

hotel_df


City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
191              asmar  35.0333  71.3581     72.27        48           0   
273       saint-pierre -21.3393  55.4781     70.57        74           0   
295            dasoguz  41.8363  59.9666     76.95        47           0   
298               hami  42.8000  93.4500     73.60         9           0   
340             kerman  30.2832  57.0788     73.26        17           0   
459            shahrud  36.4182  54.9763     78.35        14           0   
520  sao joao da barra -21.6403 -41.0511     72.54        86           0   

     Wind Speed Country        Date Hotel Name Hotel City Hotel Country  
191        7.72      AF  1622675540        NaN                           
273        5.75      RE  1622675564        NaN                           
295        8.86      TM  1622675571        NaN                           
298        8.46      CN  1622675572        NaN                           
340        3.67      IR  1622675584        NaN                           
459        4.81      IR  1622675619        NaN                           
520        9.80      BR  1622675637        NaN

In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
